<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/n_par_bn_v_1_help_tapt_pre_40_epoch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from time import time
start_time = time()

In [2]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 33.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-no

In [3]:
from datasets import load_dataset

dataset_name = "BigTMiami/amazon_MICRO_helpfulness_dataset_condensed"

dataset = load_dataset(dataset_name)

print(dataset_name)

Generating train split:   0%|          | 0/1695 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1694 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1697 [00:00<?, ? examples/s]

BigTMiami/amazon_MICRO_helpfulness_dataset_condensed


In [4]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'heads.default.3.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from adapters import ParBnConfig

reduction_factor = 16
adapter_config = ParBnConfig(reduction_factor=reduction_factor)
adapter_name = "n_par_bn_v_1_e_40_pre"

adapter_hub_name = f"{adapter_name}_adapter"
print(adapter_hub_name)

n_par_bn_v_1_e_40_pre_adapter


In [6]:
# Add a new adapter
model.add_adapter(adapter_name, config=adapter_config)

# Add head for masked language modeling
model.add_masked_lm_head(adapter_name)

# Set the adapter to be used for training
model.train_adapter(adapter_name)

print(f"Adapter Name: {adapter_name}")

Adapter Name: n_par_bn_v_1_e_40_pre


In [7]:
summary = model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
n_par_bn_v_1_e_40_pre    bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=adapter_hub_name + "_model",
    overwrite_output_dir=True,
    learning_rate=0.0004,
    per_device_train_batch_size=21,
    per_device_eval_batch_size=21,
    gradient_accumulation_steps=2, # Creates effective batch of 22 * 2 * 6 = 252
    num_train_epochs=40,
    weight_decay=0.01,
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=False,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [11]:
# from transformers import EarlyStoppingCallback

# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=10)

In [12]:
from adapters import AdapterTrainer

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["dev"],
    data_collator=pretraining_data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [13]:
train_results = trainer.train()
print(train_results)

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,6.374400,3.476433
2,2.518100,2.209916
4,2.154500,1.994234
6,2.035200,1.886460
8,1.940800,1.847935
10,1.883500,1.804095
12,1.844700,1.785273
14,1.811400,1.750868
16,1.778100,1.740444
18,1.741900,1.726425


TrainOutput(global_step=1600, training_loss=1.9259773802757263, metrics={'train_runtime': 2303.4825, 'train_samples_per_second': 29.434, 'train_steps_per_second': 0.695, 'total_flos': 1.806750603631411e+16, 'train_loss': 1.9259773802757263, 'epoch': 39.51})


In [14]:
eval_results = trainer.evaluate(dataset["test"])
print(eval_results)

{'eval_loss': 1.6682014465332031, 'eval_runtime': 18.8023, 'eval_samples_per_second': 90.095, 'eval_steps_per_second': 4.308, 'epoch': 39.51}


In [15]:
model.push_adapter_to_hub(
    adapter_hub_name,
    adapter_name,
    datasets_tag=dataset_name,
    overwrite_adapter_card=True,
)

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/n_par_bn_v_1_e_40_pre_adapter/commit/2fa987f9e9b405d6ca3b66b41366e08579dcaf0a', commit_message='Upload model', commit_description='', oid='2fa987f9e9b405d6ca3b66b41366e08579dcaf0a', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

Total Time: 41 minutes OR 2457 seconds


In [17]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
